In [335]:
using TensorKit
using TensorOperations
using MPSKit
using MPSKitModels
using Test

using WignerSymbols
using SUNRepresentations

## Vectors and Matrices

$$
\begin{split}\begin{array}{rcl}
A & : & V \rightarrow W\\
  &   & \vec{v} ↦ A(\vec{v}) \equiv \sum_j A_{ij} v_j = w_i \equiv \vec{w}
\end{array}\end{split}
$$

In [7]:
V = ℂ^2             # type as \bbC<TAB> 
W = ComplexSpace(3) # equivalent to ℂ^3
A = TensorMap(rand, Float64, W ← V) # ← as \leftarrow<TAB>
v = Tensor(rand, Float64, V)
w = A * v
w[1] ≈ A[1,1] * v[1] + A[1,2] * v[2]

true

表示法的选择主要取决于个人喜好，因为对于从左到右阅读的语言来说，从左到右似乎更传统，而在考虑数学用法时，从右到左更自然，因为矩阵通常从左到右作用于向量。在TensorKit中，通过→和←操作符支持这两种表示法，并且还提供了一个无unicode版本，默认为←。因此，以下都是等价的:

In [8]:
A = TensorMap(rand, Float64, V → W)
A = TensorMap(rand, Float64, W ← V)
A = TensorMap(rand, Float64, W, V)

TensorMap(ℂ^3 ← ℂ^2):
 0.8279779587129951   0.9322919671998565
 0.21770183546479027  0.3503372250867679
 0.8689690229051603   0.08298389457781263


## Tensors and Tensor Products

使用与上面相同的逻辑，可以通过使用张量积来推广线性映射的概念，这只不过是一个可以组合两个向量空间的操作
V和W变成一个新的向量空间V $\otimes$ W. 张量积的定义是这样的:原始向量空间的向量组合保留了自然的线性概念，也就是说，下面的等式对所有向量 $v\in V, w\in W$ 都成立, 并且对于标量 $\lambda$ 有：
$$(\lambda v) \otimes w = v \otimes (\lambda w) = \lambda (v \otimes w)$$

In [18]:
λ = rand()
(λ * v) ⊗ w ≈ v ⊗ (λ * w) ≈ λ * (v ⊗ w)

fieldnames(typeof(v⊗w))#(:data, :codom, :dom, :rowr, :colr)

v 
# 0.3490205151830481, 0.9831112140533915
w 
# 0.5677189543634762, 0.6593776595522949, 0.6607027102567581
(v⊗w)
#=
0.19814556193112184  0.23013633043713447  0.23059880031664987
0.5581308704653991   0.6482415714021404   0.6495442436088876
=#


TensorMap((ℂ^2 ⊗ ℂ^3) ← ProductSpace{ComplexSpace, 0}()):
 0.19814556193112184  0.23013633043713447  0.23059880031664987
 0.5581308704653991   0.6482415714021404   0.6495442436088876


$V\otimes W$ 空间中基向量为 $v_i\otimes w\_j$, 则这个新的空间中一个任意的向量应为：

$$t = \sum_{i_1,i_2} t_{i_1i_2} (v_{i_1} \otimes w_{i_2})$$

In [23]:
t = Tensor(rand, Float64, V ⊗ W)
t[]

2×3 StridedViews.StridedView{Float64, 2, Matrix{Float64}, typeof(identity)}:
 0.0864553  0.827808  0.1408
 0.455925   0.902138  0.115204

由于张量积可以推广到两个以上的向量空间，这最终导致了张量的一般定义，即由任意数量的向量空间的张量积建立的向量空间的元素。此外，这些对象的组成部分自然地布置在一个多维数组中，然后通过稍微滥用术语(滥用术语，在数学中，以一种形式上不正确但能简化阐述或暗示正确直觉的方式使用术语)也称为张量。

从数字列表到多维数组的组份重塑操作是线性索引 $I$ 和 和笛卡尔指标 $i_1,...,i_N$ 之间的映射。这是一个非常常见和有用的技巧，它允许将张量重新解释为向量，反之亦然。

In [24]:
LinearIndices((1:2, 1:3))

2×3 LinearIndices{2, Tuple{UnitRange{Int64}, UnitRange{Int64}}}:
 1  3  5
 2  4  6

In [27]:
collect(CartesianIndices((1:2, 1:3))) # collect to force printing

2×3 Matrix{CartesianIndex{2}}:
 CartesianIndex(1, 1)  CartesianIndex(1, 2)  CartesianIndex(1, 3)
 CartesianIndex(2, 1)  CartesianIndex(2, 2)  CartesianIndex(2, 3)

## Tensors and Multi-Linear Maps

由于向量空间的张量积本身就是一个向量空间，所以在这些向量空间之间定义线性映射也是可能的。记住(1)中线性映射的定义，列现在标记输入向量空间的组份，而行标记输出向量空间的组份。然而，输入和输出向量空间的分量本身是由来自原始向量空间的基向量的组合组成的。如果可以建立这些组合的线性顺序，则线性映射可以再次用矩阵表示：
$$
\begin{split}\begin{array}{rcl}
A & : & W_1 \otimes W_2 \otimes \cdots \otimes W_M \leftarrow 
        V_1 \otimes V_2 \otimes \cdots \otimes V_N \\
  &   & v_1 \otimes v_2 \otimes \cdots \otimes v_N \mapsto 
        A(v_1 \otimes v_2 \otimes \cdots \otimes v_N) \\
  &   & = \sum_{j_1, j_2, \cdots, j_N} A_{i_1, i_2, \cdots, i_M; j_1, j_2, \cdots, j_N}
          v_{1, j_1} \otimes v_{2, j_2} \otimes \cdots \otimes v_{N, j_N} \\
  &   & = \sum_{J} A_{I;J} v_J \\
  &   & = w_1 \otimes w_2 \otimes \cdots \otimes w_M \\
\end{array}\end{split}
$$

In [29]:
V1 = ℂ^2
V2 = ℂ^2
W1 = ℂ^2
W2 = ℂ^2

A = TensorMap(rand, Float64, W1 ⊗ W2 ← V1 ⊗ V2)

v = Tensor(rand, Float64, V1 ⊗ V2)
w = A * v
w[] ≈ reshape(reshape(A[], 4, 4) * reshape(v[], 4), 2, 2)

true

细心的读者可能已经注意到，作为矩阵的线性映射的定义与张量积向量空间中向量的定义非常相似。这不是巧合，事实上，通过考虑以下识别(同构)，两者可以很容易地识别:

$$(W \leftarrow V) \cong (W \otimes V^*)$$

In [38]:
A = TensorMap(rand, Float64, W ← V)
B = Tensor(rand, Float64, W ⊗ V')
space(A, 2) == space(B, 2)

true

## Notation

@Tensor 宏可以用来创建新的张量，使用:=赋值，或复制数据到现有的张量使用=。在后一种情况下，张量必须已经存在并且具有正确的维度，但分配的额外内存较少。


In [40]:
A = rand(2, 2, 2)
B = rand(2, 2)
@tensor C[i, j, k, l, m] := A[i, j, k] * B[l, m]
@tensor D[i] := A[i, j, j]
@tensor E[i, j, l] := A[i, j, k] * B[l, k]
size(C), size(D), size(E)

((2, 2, 2, 2, 2), (2,), (2, 2, 2))

这种表示法确实非常有用，但当希望指定以何种顺序执行成对操作时，它很快就变得笨拙起来。因此，本着同样的精神，但稍加修改，引入了NCON符号。在这个符号中，张量的指标被分配为整数，两两运算按递增顺序进行。类似地，将负整数分配给开放的腿，这决定了它们的最终位置。例如，上面的可以写成:

In [45]:
B = rand(2, 2, 2, 2)
C = rand(2, 2, 2, 2, 2)
D = rand(2, 2, 2)
E = rand(2, 2)
F = rand(2, 2)
@tensor begin
    A[-1, -2] := B[-1, 1, 2, 3] * C[3, 5, 6, 7, -2] * D[2, 4, 5] * E[1, 4] * F[6, 7]
end

2×2 Matrix{Float64}:
 3.65346  4.71469
 2.69203  3.47333

确定最优顺序是一个已知的np困难问题，因此没有算法可以有效地计算大型网络的最优顺序。尽管如此，有效的实现允许为多达30-40张量的网络找到最优顺序[Pfeifer et al，2014]，但存在其他方法可用于确定良好(不一定是最优)的收缩顺序。

In [44]:
@tensor opt=true begin
    A[i, j] := B[i, α, β, γ] * C[γ, ϵ, ζ, η, j] * D[β, δ, ϵ] * E[α, δ] * F[ζ, η]
end

2×2 Matrix{Float64}:
 1.55425  1.56018
 1.18447  1.12713

## Tensor Factorizations

线性映射允许各种各样的因式分解，这些因式分解在各种应用中都很有用。它们可以用来生成正交基，找到低秩近似，或者找到特征值和向量。在张量的情况下，已建立的矩阵分解理论可以通过将张量解释为线性映射来推广，然后将相同的分解应用于上域和定域中组成向量空间的相应矩阵划分，之后一切都延续下去。因此，唯一需要的附加信息是该分区的规范。在本节中，我们将讨论最常见的张量因子分解，但推理可以推广到任何线性映射的因子分解。

### Eigenvalue Decomposition

In [61]:
S1 = ℂ^2 ⊗ ℂ^2 ⊗ ℂ^2
S2 = ℂ^2 ⊗ ℂ^3

A = TensorMap(randn, ComplexF64, S1, S1)
D, V = eig(A)

A * V ≈ V * D

true

### Singular Value Decomposition

In [68]:
A = TensorMap(randn, ComplexF64, S1, S2)
partition = ((1, 2), (3, 4, 5))# 前 ← 后
A 
#TensorMap((ℂ^2 ⊗ ℂ^2 ⊗ ℂ^2) ← (ℂ^2 ⊗ ℂ^3))
permute(A, partition)
#TensorMap((ℂ^2 ⊗ ℂ^2) ← ((ℂ^2)' ⊗ ℂ^2 ⊗ ℂ^3))

TensorMap((ℂ^2 ⊗ ℂ^2) ← ((ℂ^2)' ⊗ ℂ^2 ⊗ ℂ^3)):
[:, :, 1, 1, 1] =
  1.7457624987467857 + 0.5185639972355347im  …  -0.6983110932727763 - 0.5811033592396013im
 -0.1621765588483261 - 0.2929613794797214im      1.6657037546758455 - 0.2317984009217034im

[:, :, 2, 1, 1] =
 -1.47205758067099 - 0.42689724855618133im  …    -0.618028194610465 + 0.34011167655490787im
 0.841991905195416 - 0.9720158542635452im      0.027250226120513917 - 1.3609152608324995im

[:, :, 1, 2, 1] =
 -0.5914379884692159 - 0.7225895380776935im  …  0.7341777608894543 - 0.5190561787253863im
  0.3013962841541692 + 0.5346623095428954im     -0.902652414196817 - 0.5834848824454189im

[:, :, 2, 2, 1] =
 -0.028513766939468665 - 1.0099826242856553im  …  0.10185389392695075 - 0.668621919196234im
    0.4378492228580174 + 1.7844128322173969im      1.8543767082661287 - 0.6990495245998324im

[:, :, 1, 1, 2] =
 -0.36310313404732947 + 0.639658101905754im   …  -0.010682798906358844 + 0.43142631065156595im
  -0.5340307835272137 - 1.00814545

In [75]:
U, S, V = tsvd(A, partition...)
permute(A, partition) ≈ U * S * V
U' * U ≈ id(domain(U))
V * V' ≈ id(codomain(V))

true

### Polar decomposition

In [76]:
A = TensorMap(randn, ComplexF64, S1, S2)
partition = ((1, 2), (3, 4, 5))
Q, P = leftorth(A, partition...; alg=Polar())
@test permute(A, partition) ≈ Q * P
@test Q * Q' ≈ id(codomain(Q))
@test (Q * Q')^2 ≈ (Q * Q')

Test Passed

### QR decomposition

In [80]:
A = TensorMap(randn, ComplexF64, S1, S2)
partition = ((1, 2), (3, 4, 5))
Q, R = leftorth(A, partition...; alg=QR())
@test permute(A, partition) ≈ Q * R
@test Q' * Q ≈ id(domain(Q))

Test Passed

最后，矩阵a的零空间是使得Ax=0的向量x的集合。这通常是通过SVD确定的，其中零空间是由对应于零奇异值的右奇异向量给出的。

In [81]:
A = TensorMap(randn, ComplexF64, S1, S2)
partition = ((1, 2, 3), (4, 5))
N = leftnull(A, partition...)
@test norm(N' * permute(A, partition)) ≈ 0 atol=1e-14
@test N' * N ≈ id(domain(N))

Test Passed

## Symmetries

$$H = -J \left (\sum_{\langle i, j \rangle} Z_i Z_j + g \sum_{i} X_i\right).$$

Here, $Z_i$ and $X_i$ are the Pauli operators acting on site $i$, and the first sum runs over pairs of nearest neighbors. This model has a global $\mathbb{Z}_2$ symmetry, as it is invariant under the transformation $UHU^\dagger = H$ where the symmetry transformation $U$ is given by a global spin flip,

$$U = \prod_i X_i.$$

Each of the operators $X$ and $Z$ acts on a local 2-dimensional complex vector space.

In [216]:
# initialize numerical data for Pauli matrices
x_mat = ComplexF64[0 1; 1 0]
z_mat = ComplexF64[1 0; 0 -1]
𝟙_mat = ComplexF64[1 0; 0 1]

# construct physical Hilbert space
V = ℂ^2

# construct the physical operators as TensorMaps
X = TensorMap(x_mat, V ← V)
Z = TensorMap(z_mat, V ← V)
𝟙 = TensorMap(𝟙_mat, V ← V)
# combine single-site operators into two-site operator
ZZ = Z ⊗ Z
H = Z ⊗ Z + X⊗𝟙 + 𝟙⊗X
U = X⊗X

@test U^2 == 𝟙⊗𝟙 #flipping all the spins twice is equivalent with leaving all spins untouched
@test U*H*U' == H # [H, U] = 0, ℤ₂={1, U}


Test Passed

In order to combine these local operators into a concrete Hamiltonian that can be used in MPSKit.jl we can make use of the convenient @mpoham macro exported by MPSKitModels.jl. For an infinite translation invariant Ising chain, we can use the following piece of code which produces the Hamiltonian in an interesting-looking form (see MPSKit.jl for details on this format).

In [97]:
lattice = InfiniteChain(1)
H = @mpoham begin
    sum(nearest_neighbours(lattice)) do (i, j)
        return ZZ{i,j}
    end + sum(vertices(lattice)) do i
        return X{i}
    end
end

@test H[rand(Int64)][1,3].data == X.data
@test H[rand(Int64)][1,1].data == Complex[1 0; 0 1]
@test H[rand(Int64)][1,2].data ≈ -√2*Z.data
@test H[rand(Int64)][2,3].data ≈ -Z.data/√2

Test Passed

$$
H = -J\sum_i S^x_iS^x_{i+1}+S^y_iS^y_{i+1}+\Delta S^z_iS^z_{i+1}.
$$
The spin operators are 2s+1-dimensional and satisfy the SU(2) commutation relations
$$
[\sigma^a_i,\sigma^b_j]=i\delta_{i,j}\sum_c \varepsilon_{abc}S^c_i
$$
Let us define the total spin
$$
S^a = \sum_i S^a_i.
$$
From a direct computation it follows that in the case where $\Delta=1$, and the model thus reduces to the Heisenberg XXX model, $H$ commutes with all $S^a, [H, S^a]=0, a=x,y,z$. 

In [133]:
Y_mat = ComplexF64[0 -im; im 0]
Y = TensorMap(Y_mat, V ← V)
Δ = 1
H = (X⊗X + Y⊗Y) + Δ*Z⊗Z 
Sx = X⊗𝟙 + 𝟙⊗X
Sy = Y⊗𝟙 + 𝟙⊗Y
Sz = Z⊗𝟙 + 𝟙⊗Z

(H*Sx == Sx*H, H*Sy == Sy*H, H*Sz == Sz*H)
#(true, ture, true)

(true, true, true)

However, when $\Delta\neq 1$ only the Z component $S^z$ commutes with $H$, $[H,S^z]=0$ Notice the difference with the Ising model where the same symmetry was present for all values of $h_x$ ($H = -\sum_{i} \sigma^z_i\sigma^z_{i+1} -h_x\sum_i\sigma^x_i$).

In [186]:
Δ = 2
θ = 2*π*rand()
H = X⊗X + Y⊗Y + Δ*Z⊗Z
(H*Sx == Sx*H, H*Sy == Sy*H, H*Sz == Sz*H, H*exp(im*θ*Sz) == exp(im*θ*Sz)*H)

(false, false, true, true)

This means that in the $\Delta=1$ case the Hamiltonian is symmetric under the full SU(2) (half integer s) or SO(3) (integer s) symmetry (see below), whereas when $\Delta\neq 1$ only an SO(2)$\simeq$ U(1) symmetry generated by $S^z$ is retained. If $H$ commutes with $S^z$ it follows that it automatically also commutes with exp($i\theta S^z$), $\theta\in[0,2\pi)$. This operator has an interpretation as a rotation around the Z-axis with an angle $\theta$.

According to Noether the Heisenberg model thus has conserved quantities associated with these operators. Regardless of $\Delta$ the Z component of the total spin is conserved, and for $\Delta=1$ all components of the total spin are conserved. In particular this means that the eigenvalue $M_z$ of $S^z$ and $S(S+1)$ of $\vec{S} \cdot \vec{S}$ are good quantum numbers to label the eigenstates of the Heisenberg Hamiltonian.

### Group Theory

#### Definition :
1. $G\times G\rightarrow G$
2. $g(hk)=(gh)k$
3. $g^{-1}g=gg^{-1}=1$

#### Examples:
- The trivial group is a group with only one element that is than automatically also the identity, and a trivial multiplication law. Above, it was denoted by {1} 

-  $\mathbb{Z}_N$. The group elements are the integers {0,1,...,N-1} and the group multiplication is addition modulo $N$. Hence it is clearly a finite group. In particular, the spin flip symmetry from above corresponds to the group $\mathbb{Z}_2$. Notice that for all $N$, $\mathbb{Z}_N$ is abelian.

- Another abelian group is $U(1)$. This group is defined as $U(1)=\left\{z\in\mathbb{C}:|z|^2 = 1\right\}$, with group multiplication the multiplication of complex numbers. Note we encountered this group in the XXZ model as being the rotations around the $Z$ axis: $\{\exp(2\pi i\theta S^z|\theta\in[0,2\pi)\}$.
  
- $SU(2)$ is the group of unimodular unitary 2 $\times$ 2 matrices:
    $$ SU(2) := \left\{U \in \mathbb{C}^{2\times 2} | 
  \det U = 1, UU^\dagger = U^\dagger U = \mathbb{I}\right\} $$

    The group multiplication is given by group multiplication. Similarly, one defines $SU(N),N\geq 2$ . Note that none of these groups are abelian.

- The 3D rotation group or special orthogonal group $SO(3)$ is the group of real 3$\times$3 orthogonal matrices with unit determinant:
    $$SO(3) := \left\{M\in\mathbb{R}^{3\times 3}|MM^T=M^TM=\mathbb{I},\det M=1\right\}$$
  Similarly, one defines $SO(N),N\geq 2$. Note that only $SO(2)$ is abelian.

### Representation Theory

A representation of a group $G$ is thus a set of matrices indexed by the group elements, $\{X_g|g\in G\}$ that multiply according to the multiplication rule of $G$:
$$X_gX_h = X_{gh}$$
Note that the identity is always mapped to the identity matrix!
We call the dimension of the matrices $X_g$ the dimension of the representation.

#### Irreducible Representations

It is clear that physical observables should not depend on any choice of basis. Therefore two representations are (unitarily) equivalent when there is a unitary basis transformation $U$ such that $X_g' =UX_gU^\dagger$. Note that $U$ is independent of $g$.

Consider again the two-dimensional regular representation of $\mathbb{Z}_2$ from above. The basis transformation
$$\begin{split}U=\frac{1}{\sqrt 2}
\begin{pmatrix}
    1 & 1\\
    1 & -1
\end{pmatrix}\end{split}$$
shows that this representation is equivalent to one where the non-trivial element of $\mathbb{Z}_2$ is represented by $U\sigma^x U^\dagger=\sigma^z$.

The crux of this example is the following. Some representations can, by an appropriate choice of basis, be brought in a form where all $X_g$ are simultaneously block-diagonal:
$$\begin{split}X_g'=UX_gU^\dagger=
\begin{pmatrix}
    \fbox{$X^1_g$} & 0 &\cdots\\
    0& \fbox{$X^2_g$} & \cdots\\
    \vdots & \vdots & \ddots
\end{pmatrix}.\end{split}$$
These blocks correspond to invariant subspaces of the representation, i.e. subspaces that transform amongst themselves under the action of the group.

An irreducible representation, irrep for short, can then be defined as a representation that can not be brought in a (non-trivial) block-diagonal form by any change of basis.

It can be shown that every finite group has a finite number of irreps. The sum of the dimensions squared is equal to the number of elements in the group: $\sum_\alpha d_\alpha^2=|G|$ where the sum is over all irreps labeled by 
$\alpha$ and $d_\alpha$ denote their respective dimensions.

One of the key questions of representation theory is what the irreps of a given group are and how the tensor product of irreps (which is in general not an irrep!) decomposes in a direct sum of irreps. The latter are sometimes known as the fusion rules. The basis transformation that reduce a given representation in a direct sum of irreps is sometimes called the Clebsch-Gordan coefficients, and are for some groups known explicitly. Before discussing the example of 
SU(2), let us first state the most important result in representation theory which is due to Schur.

[Schur’s lemma] If a matrix $Y$ commutes with all representation matrices of an irreducible representation of a group G, $X_gY=YX_g, \forall g\in G$ , then $Y$ is proportional to the identity matrix.

Example:

The answer to the questions posed above is very well understood for the case of SU(2). You probably know the answer from a previous course on quantum mechanics.

The irreps of SU(2) can be labeled by its spin, let us call it s, that takes $s=0,1/2,1,3/2,...$ values. The dimension of the spin s representation is equal to 2s+1, so there is exactly one irrep of every dimension. The spin s=0 irrep corresponds to the trivial representation.

The fusion rules can be summarized as

$$
s_1\otimes s_2 \simeq \bigoplus_{s=|s_1-s_2|}^{s_1+s_2}s.
$$

For example: $ \frac{1}{2}\otimes\frac{1}{2}\simeq 0\oplus 1 $. The Clebsch-Gordan
coefficients for $ SU(2) $ have been computed analytically, and for low-dimensional irreps have been tabulated for example
[here](https://pdg.lbl.gov/2018/reviews/rpp2018-rev-clebsch-gordan-coefs.pdf).


<a id='symmetric-tensors'></a>


### Symmetric Tensors

In physics we are often dealing with tensors that transform according to the tensor product representation of a given group $ G $. A symmetric tensor can then be understood as a tensor that transforms trivially under the action of $ G $, or more concretely under the tensor product representation $ X\otimes\bar Y\otimes\bar Z $:

![https://quantumghent.github.io/TensorTutorials/_static/SymmetricTensors/symmtens.svg](https://quantumghent.github.io/TensorTutorials/_static/SymmetricTensors/symmtens.svg)

This has strong implications for the structure of the tensor $ T $. Notice that we didn’t
assume the representations $ X,Y $ and $ Z $ to be irreducible. As we argued above, an
appropriate change of basis can bring the representations $ X,Y $ and $ Z $ in block-diagonal form where every block corresponds to an irrep of the group and every block can appear multiple times, which we call the multiplicity of an irrep in the representation. Schur’s lemma then implies that in this basis, the tensor becomes block-diagonal. In an appropriate matricization of $ T $ we can thus write $ T=\bigoplus_c B_c\otimes\mathbb{I}_c $ where the direct sum over $ c $ represents the decomposition of $ X\otimes\bar Y\otimes\bar Z $ in irreps $ c $ that can appear multiple times. In other words, the generic symmetric tensor $ T $ can be stored much more efficiently by only keeping track of the different blocks $ B_c $.

TensorKit is particularly well suited for dealing with symmetric tensors. What TensorKit
does is exactly what was described in the previous paragraph, it keeps track of the block
structure of the symmetric tensor, hereby drastically reducing the amount of memory it takes to store these objects, and is able to efficiently manipulate them by exploiting its structure to the maximum.

As a simple exercise, let us construct a rank 3 $ SU(2) $ symmetric tensor as above. For
example the spin $ 1/2 $ and spin $ 1 $ representation can be called via respectively

In [187]:
s = SU₂Space(1/2 => 1)
l = SU₂Space(1 => 1)

Rep[SU₂](1=>1)

Here, ` => 1` essentially means that we consider only one copy (direct summand) of these
representations. If we would want to consider the direct sum $ \frac{1}{2}\oplus\frac{1}{2} $ we would write

In [188]:
ss = SU₂Space(1/2 => 2)

Rep[SU₂](1/2=>2)

A symmetric tensor can now be constructed as

In [193]:
A = TensorMap(l ← s ⊗ s)

TensorMap(Rep[SU₂](1=>1) ← (Rep[SU₂](1/2=>1) ⊗ Rep[SU₂](1/2=>1))):
* Data for fusiontree FusionTree{Irrep[SU₂]}((1,), 1, (false,), ()) ← FusionTree{Irrep[SU₂]}((1/2, 1/2), 1, (false, false), ()):
[:, :, 1] =
 0.0


This tensor then has, by construction, the symmetry property that it transforms trivially
under $ 1\otimes\bar{\frac{1}{2}}\otimes\bar{\frac{1}{2}} $. The blocks can then be inspected by calling `blocks` on the tensor, and we can also check that the dimensions of the domain and codomain are as expected:

In [197]:
@assert dim(domain(A)) == 4
@assert dim(codomain(A)) == 3
blocks(A)

TensorKit.SortedVectorDict{SU2Irrep, Matrix{Float64}} with 1 entry:
  1 => [0.0;;]

We see that this tensor has one block that we can fill up with some data of our liking. Let us consider another example

In [198]:
B = TensorMap(s ← s ⊗ s)
blocks(B)

TensorKit.SortedVectorDict{SU2Irrep, Matrix{Float64}}()

This tensor does not have any blocks! This is compatible with the fact that two spin 1/2’s cannot fuse to a third spin 1/2. Finally let us consider a tensor with with more blocks:

In [202]:
C = TensorMap(ss ← ss)
blocks(C)

TensorKit.SortedVectorDict{SU2Irrep, Matrix{Float64}} with 1 entry:
  1/2 => [0.0 2.66625e-314; 2.66624e-314 0.0]